In [1]:
!git clone https://github.com/ravikanagpal/Gender_Bias_Coref_CMPUT_622

Cloning into 'Gender_Bias_Coref_CMPUT_622'...
remote: Enumerating objects: 74, done.
remote: Counting objects: 100% (74/74), done.
remote: Compressing objects: 100% (61/61), done.
remote: Total 74 (delta 19), reused 32 (delta 7), pack-reused 0
Unpacking objects: 100% (74/74), done.


In [2]:
!pip install -q neuralcoref --no-binary neuralcoref

     |████████████████████████████████| 368 kB 11.3 MB/s 
     |████████████████████████████████| 131 kB 49.1 MB/s 
     |████████████████████████████████| 79 kB 8.1 MB/s 
     |████████████████████████████████| 8.4 MB 48.8 MB/s 
     |████████████████████████████████| 127 kB 46.0 MB/s 
    Running setup.py install for neuralcoref ... done
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [3]:
!pip install -q -U spacy==2.1.0

     |████████████████████████████████| 27.7 MB 1.7 MB/s 
     |████████████████████████████████| 2.1 MB 25.1 MB/s 
     |████████████████████████████████| 3.2 MB 32.2 MB/s 
     |████████████████████████████████| 82 kB 323 kB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
en-core-web-sm 2.2.5 requires spacy>=2.2.2, but you have spacy 2.1.0 which is incompatible.


In [4]:
!python -m spacy download en

     |████████████████████████████████| 11.1 MB 12.8 MB/s 
  Created wheel for en-core-web-sm: filename=en_core_web_sm-2.1.0-py3-none-any.whl size=11074431 sha256=f3d93eeafa0cf86ae73205c6a9a19234700352eea43654f774320ce2195124f8
  Stored in directory: /tmp/pip-ephem-wheel-cache-8h03wpr6/wheels/59/4f/8c/0dbaab09a776d1fa3740e9465078bfd903cc22f3985382b496
Successfully built en-core-web-sm
  Attempting uninstall: en-core-web-sm
    Found existing installation: en-core-web-sm 2.2.5
    Uninstalling en-core-web-sm-2.2.5:
      Successfully uninstalled en-core-web-sm-2.2.5
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [5]:
import json, random
from tqdm.notebook import tqdm
from urllib.parse import quote

# Load your usual SpaCy model (one of SpaCy English models)
import spacy
from spacy import displacy
nlp = spacy.load("en_core_web_sm")

# Add neural coref to SpaCy's pipe
import neuralcoref
neuralcoref.add_to_pipe(nlp)

100%|██████████| 40155833/40155833 [00:02<00:00, 19555785.95B/s]


In [6]:
# parse each sentence and predict pronoun
import pandas as pd

data_gold = pd.read_csv("/content/Gender_Bias_Coref_CMPUT_622/data/gold_BUG.csv")
data_full = pd.read_csv("/content/Gender_Bias_Coref_CMPUT_622/data/full_BUG.csv")
data_balanced = pd.read_csv("/content/Gender_Bias_Coref_CMPUT_622/data/balanced_BUG.csv")

In [7]:
# saving the results in the csv format
def save_results(results, resultfile = "/content/Gender_Bias_Coref_CMPUT_622/output/neuralcoref.csv"):
    column_names = ["predicted gender", "sentence text", "profession", "gender","stereotype", "predicted gender", "predicted mentions", "Prediction"]
    df = pd.DataFrame(results, columns=column_names)
    # storing the results to csv file
    df.to_csv(resultfile, index=False)
    return df

In [8]:
import numpy as np

In [9]:
# This function finds coreferences clusters and check its span corresponding to occupation and gender index to get model accuracy
def evaluate(data, nlp):
  total_count = 0
  true_count = 0
  records = []
  for index, row in data.iterrows():
      s = row['sentence_text']
      if s:
        if isinstance(s, str):
          total_count += 1
          doc = nlp(row['sentence_text'])
          if doc._.has_coref:
            for cluster in doc._.coref_clusters:
              flag = False
              main = cluster.main
              main_span = main.start, main.end
              mentions_spans = [(m.start, m.end) for m in cluster.mentions \
                                if (m.start, m.end) != main_span]
              mentions = cluster.mentions
              pronouns = [sp.text for sp in mentions]
             
              if (row['profession'] in main.text) and \
                    (np.any([row['g'] in sp for sp in pronouns])):
                  true_count +=1  
                  flag = True
              records.append((row['predicted gender'], row['sentence_text'], row['profession'], row['g'], row['stereotype'], main, cluster.mentions, flag))
  save_results(records)
  print(total_count)            
  return true_count/ total_count

In [10]:
# check model accuracy on gold dataset
print(evaluate(data_gold, nlp))

1719
0.4816753926701571


In [11]:
#This function is used to calculate female vs male accuracy i.e. fairness metric delta-g
def evaluate_gender(data, nlp, gender):
  total_count = 0
  true_count = 0

  data_new = data.drop(data[(data['predicted gender'] != gender)].index)    
  for index, row in data_new.iterrows():
      s = row['sentence_text']
      if s:
        if isinstance(s, str):
          total_count += 1
          doc = nlp(row['sentence_text'])
          if doc._.has_coref:
            for cluster in doc._.coref_clusters:
              
              main = cluster.main
              main_span = main.start, main.end
              mentions_spans = [(m.start, m.end) for m in cluster.mentions \
                                if (m.start, m.end) != main_span]
              mentions = cluster.mentions
              pronouns = [sp.text for sp in mentions]
             
              if (row['profession'] in main.text) and \
                    (np.any([row['g'] in sp for sp in pronouns])):
                  true_count +=1  
                  
              
  print(total_count)            
  return true_count/ total_count


In [12]:
acc_male = evaluate_gender(data_gold, nlp, 'Male')
acc_female = evaluate_gender(data_gold, nlp, 'Female')
delta_g = acc_male - acc_female
print(delta_g)

1336
383
0.025134456934694627


In [13]:
# use this metric to calculate stereotype bias of the model
def evaluate_stereotype(data, nlp, stereotype):
  total_count = 0
  true_count = 0
 
  data_new = data.drop(data[(data['stereotype'] != stereotype)].index)
  for index, row in data_new.iterrows():
      s = row['sentence_text']
      if s:
        if isinstance(s, str):
          total_count += 1
          doc = nlp(row['sentence_text'])
          if doc._.has_coref:
            for cluster in doc._.coref_clusters:
              
              main = cluster.main
              main_span = main.start, main.end
              mentions_spans = [(m.start, m.end) for m in cluster.mentions \
                                if (m.start, m.end) != main_span]
              mentions = cluster.mentions
              pronouns = [sp.text for sp in mentions]
             
              if (row['profession'] in main.text) and \
                    (np.any([row['g'] in sp for sp in pronouns])):
                  true_count +=1  
                  
              
  print(total_count)            
  return true_count/ total_count


In [14]:
acc_antistereo = evaluate_stereotype(data_gold, nlp, -1)
acc_stereo = evaluate_stereotype(data_gold, nlp, 1)
delta_s = acc_stereo - acc_antistereo
print(delta_s)

420
864
-0.04298941798941802


In [15]:
# evaluate model on balanced dataset
print(evaluate(data_balanced, nlp))

25504
0.4126411543287328


In [16]:
# evaluate model on full dataset
print(evaluate(data_full, nlp))

105687
0.4525722179643665
